Desarrollo de modelo ML en relación ítem-ítem, esto es se toma un item, en base a que tan similar esa ese ítem al resto, se recomiendan similares. utilizando similitud del coseno de sklearn. 

In [1]:
import pandas as pd
import numpy as np

In [2]:

df_items = pd.read_csv('df_items.csv')
df_review = pd.read_csv('df_reviews.csv')
df_games = pd.read_csv('df_games.csv')

Primero limpio los df eliminando columnas inecesarias y dejando toda la informacion en un df para que pueda ser consumida por el modelo.

In [4]:
df_mini_item=df_items.drop(["playtime_forever","item_id"],axis=1)

In [6]:
merged_df = pd.merge(df_review, df_mini_item, on='user_id', how='outer')

In [8]:
merged_df.drop(['user_id', 'item_id', 'año_publicado'],axis=1, inplace=True)

In [10]:
merged_df = merged_df.rename(columns={'item_name': 'app_name'})



In [12]:
df_mini_game = df_games.drop(["año"], axis=1)

In [15]:
columnas_genero = df_mini_game.columns.difference(['app_name'])
#identifico las columnas de los generos par pder restarar el tipo de datos ya que se modifica al unir los df


In [38]:
df_funcion = pd.merge(merged_df, df_mini_game, on='app_name', how='outer')
df_funcion.head()

,recommend,sentiment_analysis,app_name,id,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,True,2.0,Counter-Strike,10.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,True,2.0,Counter-Strike,10.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,True,2.0,Counter-Strike,10.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,True,2.0,Counter-Strike,10.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,True,0.0,Counter-Strike,10.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [39]:
df_funcion = df_funcion.drop_duplicates(subset=['app_name'])

In [46]:
df_funcion = df_funcion.dropna()

In [41]:
df_funcion[columnas_genero] = df_funcion[columnas_genero].fillna(0).astype(int)
df_funcion['sentiment_analysis'] = df_funcion['sentiment_analysis'].astype(int)
#mantengo el tipo de datos correcto

In [57]:
df_funcion = df_funcion.reset_index(drop=True)


In [72]:
df_funcion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10588 entries, 0 to 10587
Data columns (total 26 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   recommend                  10588 non-null  object
 1   sentiment_analysis         10588 non-null  int32 
 2   app_name                   10588 non-null  object
 3   id                         10588 non-null  int32 
 4   Accounting                 10588 non-null  int32 
 5   Action                     10588 non-null  int32 
 6   Adventure                  10588 non-null  int32 
 7   Animation &amp; Modeling   10588 non-null  int32 
 8   Audio Production           10588 non-null  int32 
 9   Casual                     10588 non-null  int32 
 10  Design &amp; Illustration  10588 non-null  int32 
 11  Early Access               10588 non-null  int32 
 12  Education                  10588 non-null  int32 
 13  Free to Play               10588 non-null  int32 
 14  Indie 

In [82]:

df_funcion.to_csv(r'D:\Users\Natalia\Desktop\csv_limpios\df_recomendaciones.csv', index=False)
#finalmente lo guardo en un csv

Finalmete con el df limpiio solo conlos datos necesario entrenamos el modelo

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
import joblib


In [4]:
# Construir la matriz de características excluyendo las columnas de ID y título que se utilizan en la funcion
columnas_caracteristicas = df_funcion.columns.difference(['app_name', 'id']).tolist()
matriz_caracteristicas = df_funcion[columnas_caracteristicas]

In [5]:

# Calcular la similitud del coseno
cosine_similarity = cosine_similarity(matriz_caracteristicas, matriz_caracteristicas)

FUNCION

In [6]:
def obtener_recomendaciones(id_juego, cosine_similarity, df_games, df_items, df_review, n=5):
    if id_juego not in df_games['id'].values:
        return "ID de juego no encontrado en el DataFrame df_games"

    # Obtener el índice del juego según el ID en df_games
    idx = df_games.index[df_games['id'] == id_juego][0]

    # Obtener las puntuaciones de similitud para el juego dado
    sim_scores = cosine_similarity[idx]

    # Ordenar y seleccionar los juegos similares
    juego_indices = np.argsort(sim_scores)[::-1][1:n+1]

    # Obtener los item_ids de los juegos similares desde df_games
    item_ids_similares = df_games.iloc[juego_indices]['id']

    # Obtener los user_ids que han jugado juegos similares desde df_items
    user_ids_similares = df_items[df_items['item_id'].isin(item_ids_similares)]['user_id']

    # Filtrar las reviews de los usuarios que jugaron juegos similares desde df_review
    reviews_usuarios_similares = df_review[df_review['user_id'].isin(user_ids_similares)]

    # Obtener los nombres de los juegos desde df_items usando los item_ids
    juegos_recomendados = df_items[df_items['user_id'].isin(reviews_usuarios_similares['user_id'])]['item_name'].unique()[:n]

    # Crear el resultado en el formato deseado
    recomendaciones = [{"Juego " + str(i + 1): juego} for i, juego in enumerate(juegos_recomendados)]

    return recomendaciones

In [8]:
obtener_recomendaciones(352760,cosine_similarity, df_games, df_items, df_review)

[{'Juego 1': 'Half-Life 2: Lost Coast'},
 {'Juego 2': 'Counter-Strike: Source'},
 {'Juego 3': 'Half-Life 2: Deathmatch'},
 {'Juego 4': 'Half-Life Deathmatch: Source'},
 {'Juego 5': 'Half-Life 2: Episode One'}]

In [11]:
joblib.dump(similarity_matrix, 'cosine_similarity.pkl')
#guardamos el modelo para luego incorporarlo a la api

['cosine_similarity.pkl']